In [2]:
from api_24sea.core import API
from config import settings
import pandas as pd

api = API().authenticate(**settings.API_24SEA)

yacine.bel-hadj has access to https://api.24sea.eu/.
Now getting your metrics_overview table...


In [4]:
pd.DataFrame(api.get_metrics()).to_csv('metrics.csv', index=False)

In [1]:
settings.path.processed

NameError: name 'settings' is not defined